In [17]:
#Neccecary imports for the project
import numpy as np
import pandas as pd
import datetime

In [18]:
transactions_df = pd.read_parquet('../simulated-data/transactions.parquet')
transactions_df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENERIO
0,0,2024-08-16 00:00:31,596,3156,57.16,31,0,0,0
1,1,2024-08-16 00:02:10,4961,3412,81.51,130,0,0,0
2,2,2024-08-16 00:07:56,2,1365,146.00,476,0,0,0
3,3,2024-08-16 00:09:29,4128,8737,64.49,569,0,0,0
4,4,2024-08-16 00:10:34,927,9906,50.99,634,0,0,0


DATE AND TIME TRANSFORMATIONS
- TX_DURING_WEEKEND = whether a transformation occurs during a weekday (0) or weekend(1)
- TX_DURING_NIGHT = whether a transformation occurs during the day(0) or during the night(0). The night is going to be defined as hours between 0am and 6am. 

In [19]:
def is_weekend(tx_datetime):
    # Convert the string to a datetime object
    weekday = pd.to_datetime(tx_datetime).weekday()
    is_weekend = 1 if weekday > 5 else 0 
    return is_weekend

In [20]:
transactions_df['TX_DURING_WEEKEND'] = transactions_df['TX_DATETIME'].apply(is_weekend)

In [21]:
transactions_df.value_counts('TX_DURING_WEEKEND')

TX_DURING_WEEKEND
0    1380406
1     229804
Name: count, dtype: int64

In [22]:
def is_night(tx_datetime):
    hour = tx_datetime.hour
    is_night = 1 if hour <= 6 else 0
    return is_night


In [23]:
transactions_df['TX_DURING_NIGHT'] = transactions_df['TX_DATETIME'].apply(is_night)

In [24]:
transactions_df.value_counts('TX_DURING_NIGHT')

TX_DURING_NIGHT
0    1330092
1     280118
Name: count, dtype: int64

In [25]:
transactions_df[transactions_df['TX_TIME_DAYS'] >= 40]

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENERIO,TX_DURING_WEEKEND,TX_DURING_NIGHT
384041,384041,2024-09-25 00:00:20,1182,4477,6.49,3456020,40,0,0,0,1
384042,384042,2024-09-25 00:00:59,2899,1212,53.34,3456059,40,0,0,0,1
384043,384043,2024-09-25 00:02:17,1843,7238,92.51,3456137,40,0,0,0,1
384044,384044,2024-09-25 00:02:46,1210,1598,25.46,3456166,40,0,0,0,1
384045,384045,2024-09-25 00:02:59,4526,4677,100.32,3456179,40,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
1610205,1610205,2025-01-30 23:55:00,1774,6994,21.45,14514900,167,0,0,0,0
1610206,1610206,2025-01-30 23:55:17,4161,9595,23.42,14514917,167,0,0,0,0
1610207,1610207,2025-01-30 23:56:14,3179,2,46.73,14514974,167,0,0,0,0
1610208,1610208,2025-01-30 23:56:27,4399,3644,77.21,14514987,167,0,0,0,0


CUSTOMER ID TRANSFORMATIONS
- For a customer we will generate another ID related RFM(Recency, Frequency, Monetary). We will compute two of these features over three times window (1, 7, 30) 
- This will generate six new features. Note that these time windows could later be optimized along with the models using a model selection procedure 

In [26]:
def get_customer_spending_behaviour_features(customer_transactions, window_size_indays = [1,7,30]):
    #Sort the transactions by date
    customer_transactions = customer_transactions.sort_values('TX_DATETIME')
    #Set the index as the TX_DATETIME
    customer_transactions.index = customer_transactions.TX_DATETIME
    
    #For each window size
    for window_size in window_size_indays:
        # Compute the sum of the transaction amounts and the number of transactions for the given window size
        SUM_AMOUNT_TX_WINDOW = customer_transactions['TX_AMOUNT'].rolling(str(window_size) + 'd').sum()
        NB_TX_WINDOW=customer_transactions['TX_AMOUNT'].rolling(str(window_size)+'d').count()

        # Compute the average transaction amount given window
        AVG_AMOUNT_TX_WINDOW = SUM_AMOUNT_TX_WINDOW / NB_TX_WINDOW

        # Save feature values
        customer_transactions['CUSTOMER_ID_NB_TX' + str(window_size) + 'DAY_WINDOW'] = list(NB_TX_WINDOW)
        customer_transactions['CUSTOMER_ID_AVG_AMOUNT' + str(window_size) + 'DAY_WINDOW'] = list(AVG_AMOUNT_TX_WINDOW)

    #Reindex according to transaction IDs
    customer_transactions.index=customer_transactions.TRANSACTION_ID
        
    # And return the dataframe with the new features
    return customer_transactions



In [27]:
#Example for a given customer
spending_behaviour_customer_0 = get_customer_spending_behaviour_features(transactions_df[transactions_df['CUSTOMER_ID']==0])
spending_behaviour_customer_0

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENERIO,TX_DURING_WEEKEND,TX_DURING_NIGHT,CUSTOMER_ID_NB_TX1DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT1DAY_WINDOW,CUSTOMER_ID_NB_TX7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT7DAY_WINDOW,CUSTOMER_ID_NB_TX30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT30DAY_WINDOW
TRANSACTION_ID,,,,,,,,,,,,,,,,,
1758,1758,2024-08-16 07:19:05,0,6076,123.59,26345,0,0,0,0,0,1.0,123.590000,1.0,123.59000,1.0,123.590000
8275,8275,2024-08-16 18:00:16,0,858,77.34,64816,0,0,0,0,0,2.0,100.465000,2.0,100.46500,2.0,100.465000
8640,8640,2024-08-16 19:02:02,0,6698,46.51,68522,0,0,0,0,0,3.0,82.480000,3.0,82.48000,3.0,82.480000
12169,12169,2024-08-17 08:51:06,0,6569,54.72,118266,1,0,0,0,0,3.0,59.523333,4.0,75.54000,4.0,75.540000
15764,15764,2024-08-17 14:05:38,0,7707,63.30,137138,1,0,0,0,0,4.0,60.467500,5.0,73.09200,5.0,73.092000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1571573,1571573,2025-01-26 21:16:25,0,3383,71.66,14159785,163,0,0,1,0,4.0,62.725000,20.0,60.82000,68.0,69.463824
1578306,1578306,2025-01-27 14:25:10,0,9373,43.61,14221510,164,0,0,0,0,4.0,71.882500,20.0,58.76000,68.0,69.358824
1595334,1595334,2025-01-29 11:07:59,0,1536,94.92,14382479,166,0,0,0,0,1.0,94.920000,15.0,58.49800,64.0,69.245469


customer_0 has mean_amount = 62.262521 and mean_nb_tx_per_day = 2.179533. we can clearly see this values can be closely observed especially in the window of 30 days

In [28]:
%time transactions_df=transactions_df.groupby('CUSTOMER_ID').apply(lambda x: get_customer_spending_behaviour_features(x, window_size_indays=[1,7,30]))
transactions_df = transactions_df.sort_values('TX_DATETIME').reset_index(drop=True)

<timed exec>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


CPU times: user 6.53 s, sys: 168 ms, total: 6.7 s
Wall time: 6.75 s


In [29]:
transactions_df

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENERIO,TX_DURING_WEEKEND,TX_DURING_NIGHT,CUSTOMER_ID_NB_TX1DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT1DAY_WINDOW,CUSTOMER_ID_NB_TX7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT7DAY_WINDOW,CUSTOMER_ID_NB_TX30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT30DAY_WINDOW
0,0,2024-08-16 00:00:31,596,3156,57.16,31,0,0,0,0,1,1.0,57.160000,1.0,57.160000,1.0,57.160000
1,1,2024-08-16 00:02:10,4961,3412,81.51,130,0,0,0,0,1,1.0,81.510000,1.0,81.510000,1.0,81.510000
2,2,2024-08-16 00:07:56,2,1365,146.00,476,0,0,0,0,1,1.0,146.000000,1.0,146.000000,1.0,146.000000
3,3,2024-08-16 00:09:29,4128,8737,64.49,569,0,0,0,0,1,1.0,64.490000,1.0,64.490000,1.0,64.490000
4,4,2024-08-16 00:10:34,927,9906,50.99,634,0,0,0,0,1,1.0,50.990000,1.0,50.990000,1.0,50.990000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1610205,1610205,2025-01-30 23:55:00,1774,6994,21.45,14514900,167,0,0,0,0,3.0,16.046667,13.0,11.818462,64.0,27.084219
1610206,1610206,2025-01-30 23:55:17,4161,9595,23.42,14514917,167,0,0,0,0,2.0,14.440000,23.0,20.338261,123.0,21.500650
1610207,1610207,2025-01-30 23:56:14,3179,2,46.73,14514974,167,0,0,0,0,4.0,58.480000,20.0,66.644500,68.0,70.123971
1610208,1610208,2025-01-30 23:56:27,4399,3644,77.21,14514987,167,0,0,0,0,2.0,71.125000,9.0,87.168889,46.0,87.769348


TERMINAL ID TRANSFORMATIONS
- The main goal will be to extract a risk score, that assesses the exposure of a given terminal ID to fraudulent transactions.
- Instead of windowing precede a given transformation we have to shift back by a delay period. In real scenerios, fraudelent terminals are only discovered after a fraud investigation or customer complains. 

NB_FRAUD_WINDOW=NB_FRAUD_DELAY_WINDOW-NB_FRAUD_DELAY
NB_TX_WINDOW=NB_TX_DELAY_WINDOW-NB_TX_DELAY

RISK_WINDOW=NB_FRAUD_WINDOW/NB_TX_WINDOW

In [30]:
def get_count_risk_rolling_window(terminal_transactions, delay_period = 7, window_size_indays = [1,7,30], feature = "TERMINAL_ID" ):
    terminal_transactions = terminal_transactions.sort_values('TX_DATETIME')
    terminal_transactions.index = terminal_transactions.TX_DATETIME

    NB_FRAUD_DELAY = terminal_transactions['TX_FRAUD'].rolling(str(delay_period) + 'd').sum()
    NB_TX_DELAY = terminal_transactions['TX_FRAUD'].rolling(str(delay_period) + 'd').count()

    for window_size in window_size_indays:
    
        NB_FRAUD_DELAY_WINDOW=terminal_transactions['TX_FRAUD'].rolling(str(delay_period+window_size)+'d').sum()
        NB_TX_DELAY_WINDOW=terminal_transactions['TX_FRAUD'].rolling(str(delay_period+window_size)+'d').count()
    
        NB_FRAUD_WINDOW=NB_FRAUD_DELAY_WINDOW-NB_FRAUD_DELAY
        NB_TX_WINDOW=NB_TX_DELAY_WINDOW-NB_TX_DELAY
    
        RISK_WINDOW=NB_FRAUD_WINDOW/NB_TX_WINDOW
        
        terminal_transactions[feature+'_NB_TX_' + str(window_size)+'DAY_WINDOW']=list(NB_TX_WINDOW)
        terminal_transactions[feature+'_RISK_' +str(window_size)+'DAY_WINDOW']=list(RISK_WINDOW)
        
    terminal_transactions.index=terminal_transactions.TRANSACTION_ID
    
    # Replace NA values with 0 (all undefined risk scores where NB_TX_WINDOW is 0) 
    terminal_transactions.fillna(0,inplace=True)
    
    return terminal_transactions



In [31]:
transactions_df[transactions_df.TX_FRAUD==1]

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENERIO,TX_DURING_WEEKEND,TX_DURING_NIGHT,CUSTOMER_ID_NB_TX1DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT1DAY_WINDOW,CUSTOMER_ID_NB_TX7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT7DAY_WINDOW,CUSTOMER_ID_NB_TX30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT30DAY_WINDOW
3527,3527,2024-08-16 10:17:43,3774,3059,225.41,37063,0,1,1,0,0,3.0,158.073333,3.0,158.073333,3.0,158.073333
3888,3888,2024-08-16 10:49:01,4836,5250,315.95,38941,0,1,3,0,0,2.0,208.015000,2.0,208.015000,2.0,208.015000
5789,5789,2024-08-16 13:31:48,4944,6050,222.26,48708,0,1,1,0,0,2.0,127.605000,2.0,127.605000,2.0,127.605000
6549,6549,2024-08-16 14:42:02,4625,9102,226.40,52922,0,1,1,0,0,4.0,167.165000,4.0,167.165000,4.0,167.165000
6811,6811,2024-08-16 15:07:17,4836,3297,410.30,54437,0,1,3,0,0,3.0,275.443333,3.0,275.443333,3.0,275.443333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1609562,1609562,2025-01-30 19:35:26,3552,4387,34.58,14499326,167,1,2,0,0,7.0,34.024286,23.0,40.937826,81.0,43.482099
1609609,1609609,2025-01-30 19:45:32,962,1560,47.85,14499932,167,1,2,0,0,6.0,32.601667,17.0,28.847059,55.0,26.643091
1609696,1609696,2025-01-30 20:07:38,2409,4537,138.35,14501258,167,1,3,0,0,6.0,91.603333,26.0,71.693846,84.0,47.709762
1609708,1609708,2025-01-30 20:09:29,4195,5607,24.32,14501369,167,1,2,0,0,3.0,14.176667,24.0,21.332500,108.0,25.840463


In [32]:
#lets compute these six features for the first terminal ID containing at least one fraud = 3059
get_count_risk_rolling_window(transactions_df[transactions_df['TERMINAL_ID']==3059], delay_period = 7, window_size_indays = [1,7,30])


,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENERIO,TX_DURING_WEEKEND,...,CUSTOMER_ID_NB_TX7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT7DAY_WINDOW,CUSTOMER_ID_NB_TX30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
TRANSACTION_ID,,,,,,,,,,,,,,,,,,,,,
3527,3527,2024-08-16 10:17:43,3774,3059,225.41,37063,0,1,1,0,...,3.0,158.073333,3.0,158.073333,0.0,0.0,0.0,0.0,0.0,0.0
4732,4732,2024-08-16 11:59:14,55,3059,36.28,43154,0,0,0,0,...,2.0,35.670000,2.0,35.670000,0.0,0.0,0.0,0.0,0.0,0.0
16216,16216,2024-08-17 14:47:34,4879,3059,105.00,139654,1,0,0,0,...,10.0,76.010000,10.0,76.010000,0.0,0.0,0.0,0.0,0.0,0.0
18249,18249,2024-08-17 19:08:10,2263,3059,90.89,155290,1,0,0,0,...,7.0,50.458571,7.0,50.458571,0.0,0.0,0.0,0.0,0.0,0.0
26512,26512,2024-08-18 15:44:49,4879,3059,58.51,229489,2,0,0,1,...,14.0,71.070000,14.0,71.070000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1575907,1575907,2025-01-27 11:01:05,2263,3059,59.26,14209265,164,0,0,0,...,15.0,62.733333,74.0,55.855946,3.0,0.0,10.0,0.0,33.0,0.0
1584450,1584450,2025-01-28 09:05:13,2508,3059,28.88,14288713,165,0,0,0,...,22.0,35.979091,97.0,32.995979,1.0,0.0,11.0,0.0,34.0,0.0
1604244,1604244,2025-01-30 10:11:54,1483,3059,6.72,14465514,167,0,0,0,...,23.0,7.516957,93.0,7.975484,2.0,0.0,12.0,0.0,36.0,0.0


In [33]:
%time transactions_df=transactions_df.groupby('TERMINAL_ID').apply(lambda x: get_count_risk_rolling_window(x, delay_period=7, window_size_indays=[1,7,30], feature="TERMINAL_ID"))
transactions_df=transactions_df.sort_values('TX_DATETIME').reset_index(drop=True)

<timed exec>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


CPU times: user 14.2 s, sys: 252 ms, total: 14.4 s
Wall time: 14.5 s


In [34]:
transactions_df

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENERIO,TX_DURING_WEEKEND,...,CUSTOMER_ID_NB_TX7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT7DAY_WINDOW,CUSTOMER_ID_NB_TX30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
0,0,2024-08-16 00:00:31,596,3156,57.16,31,0,0,0,0,...,1.0,57.160000,1.0,57.160000,0.0,0.0,0.0,0.0,0.0,0.000000
1,1,2024-08-16 00:02:10,4961,3412,81.51,130,0,0,0,0,...,1.0,81.510000,1.0,81.510000,0.0,0.0,0.0,0.0,0.0,0.000000
2,2,2024-08-16 00:07:56,2,1365,146.00,476,0,0,0,0,...,1.0,146.000000,1.0,146.000000,0.0,0.0,0.0,0.0,0.0,0.000000
3,3,2024-08-16 00:09:29,4128,8737,64.49,569,0,0,0,0,...,1.0,64.490000,1.0,64.490000,0.0,0.0,0.0,0.0,0.0,0.000000
4,4,2024-08-16 00:10:34,927,9906,50.99,634,0,0,0,0,...,1.0,50.990000,1.0,50.990000,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1610205,1610205,2025-01-30 23:55:00,1774,6994,21.45,14514900,167,0,0,0,...,13.0,11.818462,64.0,27.084219,0.0,0.0,8.0,0.0,32.0,0.000000
1610206,1610206,2025-01-30 23:55:17,4161,9595,23.42,14514917,167,0,0,0,...,23.0,20.338261,123.0,21.500650,0.0,0.0,9.0,0.0,29.0,0.000000
1610207,1610207,2025-01-30 23:56:14,3179,2,46.73,14514974,167,0,0,0,...,20.0,66.644500,68.0,70.123971,0.0,0.0,3.0,0.0,18.0,0.000000
1610208,1610208,2025-01-30 23:56:27,4399,3644,77.21,14514987,167,0,0,0,...,9.0,87.168889,46.0,87.769348,0.0,0.0,10.0,0.0,35.0,0.028571


In [35]:
transactions_df.columns

Index(['TRANSACTION_ID', 'TX_DATETIME', 'CUSTOMER_ID', 'TERMINAL_ID',
       'TX_AMOUNT', 'TX_TIME_SECONDS', 'TX_TIME_DAYS', 'TX_FRAUD',
       'TX_FRAUD_SCENERIO', 'TX_DURING_WEEKEND', 'TX_DURING_NIGHT',
       'CUSTOMER_ID_NB_TX1DAY_WINDOW', 'CUSTOMER_ID_AVG_AMOUNT1DAY_WINDOW',
       'CUSTOMER_ID_NB_TX7DAY_WINDOW', 'CUSTOMER_ID_AVG_AMOUNT7DAY_WINDOW',
       'CUSTOMER_ID_NB_TX30DAY_WINDOW', 'CUSTOMER_ID_AVG_AMOUNT30DAY_WINDOW',
       'TERMINAL_ID_NB_TX_1DAY_WINDOW', 'TERMINAL_ID_RISK_1DAY_WINDOW',
       'TERMINAL_ID_NB_TX_7DAY_WINDOW', 'TERMINAL_ID_RISK_7DAY_WINDOW',
       'TERMINAL_ID_NB_TX_30DAY_WINDOW', 'TERMINAL_ID_RISK_30DAY_WINDOW'],
      dtype='object')

In [37]:
#save the new features to a parquet file
transactions_df.to_parquet('../simulated-data-transformed/transactions.parquet')